### This is a program to simulate tree based Pseudo-Least Recently Used(PLRU) replacement policy for an n-way set assoiative  cache

In [23]:
import math
import numpy as np

In [30]:
# create a node class for the bit tree that is maintained for each set
class Node(object):  
    def __init__(self):
        self.bit = 0
        self.left = None
        self.right = None

##### Create a root node
root = Node()
##### Create a tree with root as root node
tree = Tree(root)
##### Build tree with height = logbase2 (n-ways) by calling build_tree function
tree.build_tree(root,4)

In [49]:
class Tree(object): 
    def __init__(self,root):
        self.root = root
        self.height = 0
    
    def insert(self, root, height):
        it = height-1
        if it == 0:
            return
        root.left = Node()
        root.right = Node()
        self.insert(root.left,it)
        self.insert(root.right,it)
    
    def build_tree(self, root, ways):
        self.height = int(np.log2(ways))
        self.insert(root, self.height)
        return root

##### build cache args: cache_size in KB, no of words per line, no of ways
cache = CacheBuilder(4,16,4)
##### create cache by calling create_cache() method
cache.create_cache()
##### access cache through the cache_dict dictionary. 
cache.cache_dict
The dictionary has all sets and each set has one tree. Each leaf node of tree is linked to two lines of the set

In [109]:
class CacheBuilder(object):
    # 1 word = 4bytes = 32bits
    # cache size in KB (kilobytes)
    # n = no of ways
    # line_size = no of words in one line 
    # i.e line_size is the max no of intigers that can be in one line 
    # assuming one int = 32 bits or 4 bytes or 1 word
    def __init__(self, cache_size, line_size, n):
        self.ways = n
        self.line_size = line_size
        self.cache_size = cache_size
        self.no_of_lines = (cache_size*1024)/(line_size*4)
        if (n<=self.no_of_lines):
            self.no_of_sets = self.no_of_lines/n
        else:
            self.no_of_sets = 0
            print "no of ways cannot be less then total no of lines"
        self.cache_dict = {}
    
    def create_cache(self):
        if(self.no_of_sets == 0):
            print "cache cannot be built, reason: no of sets cannot be less then no of lines"
            return
        # create a tree for each set
        for i in range(self.no_of_sets):
            self.cache_dict[i] = {}
            root = Node()
            tree = Tree(root)
            tree.build_tree(root, self.ways)
            self.cache_dict[i]["tree"] = tree
            #create lines
            lines = []
            for j in range(self.ways):
                line = []
                lines.append(line)
            self.cache_dict[i]["lines"] = lines
        